In [1]:
import numpy as np
import pandas as pd
import scipy.sparse
import xgboost as xgb
import mix_pandas as mix
import predict as predict_mix
import db_column_name as db

%matplotlib inline
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

from pylab import rcParams
rcParams['figure.figsize'] = 10, 7


cn = db.ColumnName()

target_minT = pd.read_csv('./data/31286_103.csv')
mix.set_index_date(target_minT, cn.date)

X = pd.read_csv('./data/character_31286.csv')
mix.set_index_date(X, cn.date)

X = X.drop([cn.point], axis=1)
X = X[[x for x in X.columns if 'avg' in x or 
       x == cn.offset]]
    
X = mix.mean_day(X)
target_minT.index = target_minT.index.round('D')

X = X.drop([cn.offset], axis=1)

target_minT = target_minT.reindex(X.index)
target_minT = mix.clean(target_minT)
X = X.reindex(target_minT.index)
X = mix.clean(X)

target_minT = mix.winsorized(target_minT, cn.value, [0.05, 0.95])
X = X.reindex(target_minT.index)
print(X.shape)


(1517, 54)


In [5]:
from sklearn.linear_model import LinearRegression, Lasso, RidgeCV

predict = predict_mix.predict_model_split(LinearRegression(), X, target_minT, cn.value, 5)
for train, test in predict:
    print("Train size {}".format(train.shape[0]))
    predict_mix.print_mean(train[[cn.value]], train[['prediction']])
    print("Test size {}".format(test.shape[0]))
    predict_mix.print_mean(test[[cn.value]], test[['prediction']])
    print()

Train size 257
Mean squared error 6.76671
Mean absolute error 2.09928
Median absolute error 1.77030
Test size 252
Mean squared error 65801119017.67209
Mean absolute error 16162.06521
Median absolute error 2.31203

Train size 509
Mean squared error 8.45392
Mean absolute error 2.27194
Median absolute error 1.85349
Test size 252
Mean squared error 12.89042
Mean absolute error 2.76712
Median absolute error 2.28646

Train size 761
Mean squared error 9.38013
Mean absolute error 2.36038
Median absolute error 1.86044
Test size 252
Mean squared error 194.50093
Mean absolute error 10.06967
Median absolute error 5.75510

Train size 1013
Mean squared error 21.05453
Mean absolute error 3.51105
Median absolute error 2.83011
Test size 252
Mean squared error 757.30941
Mean absolute error 21.54150
Median absolute error 16.92616

Train size 1265
Mean squared error 45.19330
Mean absolute error 5.66850
Median absolute error 5.35384
Test size 252
Mean squared error 109.50787
Mean absolute error 8.57708
Med

In [8]:
lasso = Lasso(alpha=5, tol=0.001)

predict = predict_mix.predict_model_split(lasso, X, target_minT, cn.value, 5)
for train, test in predict:
    print("Train size {}".format(train.shape[0]))
    predict_mix.print_mean(train[[cn.value]], train[['prediction']])
    print("Test size {}".format(test.shape[0]))
    predict_mix.print_mean(test[[cn.value]], test[['prediction']])
    print()
    # test.plot(style=".")

/usr/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/usr/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/usr/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/usr/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting dat

Train size 257
Mean squared error 10.91087
Mean absolute error 2.61752
Median absolute error 2.06419
Test size 252
Mean squared error 15.99891
Mean absolute error 3.06126
Median absolute error 2.47904

Train size 509
Mean squared error 12.03935
Mean absolute error 2.69571
Median absolute error 2.03059
Test size 252
Mean squared error 16.31425
Mean absolute error 3.06284
Median absolute error 2.32801

Train size 761
Mean squared error 13.09677
Mean absolute error 2.78092
Median absolute error 2.13238
Test size 252
Mean squared error 236.71356
Mean absolute error 11.10011
Median absolute error 6.42727

Train size 1013
Mean squared error 27.93149
Mean absolute error 4.03077
Median absolute error 3.31276
Test size 252
Mean squared error 587.24577
Mean absolute error 19.92157
Median absolute error 17.97821

Train size 1265
Mean squared error 50.65503
Mean absolute error 6.05357
Median absolute error 5.80524
Test size 252
Mean squared error 49.88616
Mean absolute error 5.75592
Median absolut

/usr/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [9]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression, Lasso
from stepwise import stepwise_selection

selectColumns = stepwise_selection(X, target_minT)
X_select = X.loc[:, selectColumns]

X_select.head()

/home/cheri/Документы/Course-work/stepwise.py:34: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  best_feature = new_pval.argmin()


Add  avg t2                         with p-value 0.0
Add  avg rh2                        with p-value 6.31268e-40
Add  avg v850                       with p-value 2.38758e-10
Add  avg rot500                     with p-value 2.09621e-06
Add  avg t500                       with p-value 0.00295242


,avg t2,avg rh2,avg v850,avg rot500,avg t500
2013-01-03,-28.59545,56.20200,-13.865800,-0.000013,-40.96270
2013-01-04,-26.42975,57.46880,-10.749225,-0.000036,-40.88725
2013-01-05,-29.51275,57.06995,-14.566150,-0.000008,-41.31615
2013-01-06,-31.34385,56.45440,-15.012850,0.000060,-42.62750
2013-01-08,-30.32600,60.43055,-15.454200,0.000041,-42.74250


In [10]:
predict = predict_mix.predict_model_split(LinearRegression(), X_select, target_minT, cn.value, 5)
for train, test in predict:
    print("Train size {}".format(train.shape[0]))
    predict_mix.print_mean(train[[cn.value]], train[['prediction']])
    print("Test size {}".format(test.shape[0]))
    predict_mix.print_mean(test[[cn.value]], test[['prediction']])
    print()
    
# 16.3 8.4 16.5 12.3 15.3  

Train size 257
Mean squared error 10.55780
Mean absolute error 2.55267
Median absolute error 1.96449
Test size 252
Mean squared error 13.12892
Mean absolute error 2.75376
Median absolute error 2.06963

Train size 509
Mean squared error 11.22314
Mean absolute error 2.57872
Median absolute error 2.03461
Test size 252
Mean squared error 15.48847
Mean absolute error 2.95468
Median absolute error 2.31521

Train size 761
Mean squared error 12.51519
Mean absolute error 2.69526
Median absolute error 2.13538
Test size 252
Mean squared error 297.79322
Mean absolute error 12.27145
Median absolute error 6.72305

Train size 1013
Mean squared error 35.94675
Mean absolute error 4.64411
Median absolute error 3.81824
Test size 252
Mean squared error 419.76803
Mean absolute error 17.47460
Median absolute error 17.55335

Train size 1265
Mean squared error 51.35807
Mean absolute error 6.13516
Median absolute error 5.93107
Test size 252
Mean squared error 44.99205
Mean absolute error 5.63685
Median absolut